## Cross-account invoke example
### Invoke a Sagemaker Endpoint in another AWS account from an app (or notebook instance) that is in a private VPC (no internet) using cross-account assume role and private link

#### This example code simulates an app that runs inside a VPC that has no internet gateway, i.e. no inbound / outbound internet connectivity. It demonstrates two things:
1. How you can use AWS Private Link (interface endpoints) for STS and Sagemaker to talk to these services over private networking
2. How you can invoke an endpoint running on another AWS account

#### To run this example successfully, you need to complete the following pre-requisites:
1. Have two AWS accounts, Acc A (consumer) and Acc B (provider). Acc B will host the Sagemaker endpoint, which Acc A will invoke
2. Go to Acc B and create the "house price prediction by city" Sagemaker Endpoint using [this](https://github.com/aws/amazon-sagemaker-examples/blob/main/advanced_functionality/multi_model_xgboost_home_value/xgboost_multi_model_endpoint_home_value.ipynb) notebook
3. Create an IAM role in Acc B that grants Acc A permissions to call Sagemaker in Acc B (it's done by creating a new role with "Trusted Entity" = Account). Attach a trust policy to the role that allows Acc A to assume this role
4. Create an IAM role in Acc A that has permissions to call Sagemaker and also a custom policy that enables Acc A to assume the IAM role created in step 3
5. Create a VPC endpoint for Sagemaker-runtime in Acc A, ensure private dns is enabled and VPC endpoint is in the same subnet and same security group as the Sagemaer notebook instance running this notebook 
6. Create a VPC endpoint for sts in Acc A, ensure private dns is enabled and VPC endpoint is in the same subnet and same security group as the Sagemaer notebook instance running this notebook 
7. Ensure that the Sagemaker notebook instance running this notebook has the IAM role attached that you craeted in step 4



## Run this code in a Sagemaker Notebook in Acc A (consumer AWS account)

## Imports

In [ ]:
import numpy as np
import pandas as pd
import boto3
import sagemaker
print("Sagemaker version: "+sagemaker.__version__)

## Setup the STS client
Notice that I'm explictly specifying region and endpoint_url. This is because, by default, Boto3 sdk uses the global DNS name for STS, which won't go through the Interface endpoint you created

In [22]:
sts = boto3.client("sts", 
                   region_name="us-east-1",
                   endpoint_url="https://sts.us-east-1.amazonaws.com")

In [23]:
sts = boto3.client("sts", 
                   endpoint_url="https://sts.us-east-1.amazonaws.com")

### This is the cross account IAM role you created earlier

In [24]:
CROSS_ACCOUNT_ASSUME_ROLE = "arn:aws:iam::[Acc B Id]:role/Sagemaker-cross-acc-role"

In [25]:
response = sts.assume_role(RoleArn=CROSS_ACCOUNT_ASSUME_ROLE, 
                           RoleSessionName="InferenceCrossAccountAccessDemo",
                                     ExternalId="[Acc B Id]")

## Setup Sagemaker client and invoke the endpoint in the other account

In [26]:
sm = boto3.client(
    "sagemaker-runtime",
    aws_access_key_id=response["Credentials"]["AccessKeyId"],
    aws_secret_access_key=response["Credentials"]["SecretAccessKey"],
    aws_session_token=response["Credentials"]["SessionToken"] 
)

In [27]:
ENDPOINT_NAME = "mme-xgboost-housing"
INPUT = "1972, 2844, 3, 2.5, 0.74, 2"
TARGET_MODEL="NewYork_NY.tar.gz"
MIME_TYPE="text/csv"
response = sm.invoke_endpoint(
    EndpointName=ENDPOINT_NAME, 
    ContentType=MIME_TYPE,
    TargetModel=TARGET_MODEL,
    Body=INPUT
    )

In [28]:
result = response["Body"].read()

In [29]:
value = float(result.decode("utf-8").replace("[","").replace("]",""))

In [ ]:
print("${:,.2f}\n".format(value))